In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#!pip freeze

In [ ]:
#!pip install imutils
#!pip freeze

#!pip install -r '../input/requirement/requirements.txt'
#!pip install h5py==2.10.0
#!pip install tensorflow==2.1.0
#!pip install keras==2.3.1

# **Train.py**

In [ ]:
from PIL import Image

import pandas as pd
import numpy as np
import os
#import keras
import random
import cv2
from tqdm import tqdm
import math
#import seaborn as sns

#from keras.preprocessing import image
#from keras.models import load_model
#from imutils import paths

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.utils import class_weight

#import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout

from tensorflow.keras.applications import DenseNet121, DenseNet201,ResNet50,ResNet152V2

from tensorflow.keras.applications.densenet import preprocess_input

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array

from tensorflow.keras.models import Model, load_model

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard, LearningRateScheduler

import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options = 
                         tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)
# device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

import os

BASE_DIR = '../input/ranzcr-clip-catheter-line-classification'




train= pd.read_csv(os.path.join(BASE_DIR,'train.csv'))


train['StudyInstanceUID'] = train['StudyInstanceUID'].apply(lambda x: x +".jpg")



print(int(len(train)*0.9))
train_split = int(len(train)*0.9)
val_split=int(len(train)*0.1)
#test_split= int(len(train)*0.2)

train_data = train[:train_split]

val_data = train[train_split:train_split+val_split]

#test_data = train[train_split+val_split:]



target_cols = ['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal', 'NGT - Abnormal', 
               'NGT - Borderline', 'NGT - Incompletely Imaged', 'NGT - Normal', 'CVC - Abnormal',
               'CVC - Borderline', 'CVC - Normal', 'Swan Ganz Catheter Present']

'''
for c in target_cols:
    plt.hist(train_data[c].values)
    plt.title(f'target: {c}')
    plt.show()
'''  
  
# random labels_dict
labels_dict = train[target_cols].value_counts().to_dict()



cls_weight = {}
for i in range(len(target_cols)):
    single_cls_weight={}
    for j in range(len(np.unique(train_data[target_cols[i]]))):
        single_cls_weight[np.unique(train_data[target_cols[i]])[j]]=class_weight.compute_class_weight('balanced',
                                 np.unique(train_data[target_cols[i]]),
                                 train_data[target_cols[i]])[j]
    cls_weight[i] = single_cls_weight
    
    
    
#print(cls_weight)


trainAug = ImageDataGenerator(
  rescale=1/255.0,
  rotation_range=30,
  zoom_range=0.2,
  width_shift_range=0.1,
  height_shift_range=0.1,
  shear_range=0.05,
  horizontal_flip=True,
  vertical_flip=True,
  fill_mode="nearest")

validationAug = ImageDataGenerator(rescale=1./255)


trainGen = trainAug.flow_from_dataframe(
    train_data, directory=os.path.join(BASE_DIR,'train'), x_col='StudyInstanceUID', y_col=target_cols,
    class_mode='raw',
    weight_col=None, target_size=(256, 256), color_mode='rgb',
    classes=None, batch_size=64, shuffle=True,
    seed=None, save_to_dir=None, save_prefix='',
    save_format='jpg', subset=None, interpolation='nearest',
    validate_filenames=True
)

valGen = validationAug.flow_from_dataframe(
    val_data, directory=os.path.join(BASE_DIR,'train'), x_col='StudyInstanceUID', y_col=target_cols,
    class_mode='raw',
    weight_col=None, target_size=(256, 256), color_mode='rgb',
    classes=None, batch_size=64, shuffle=True,
    seed=None, save_to_dir=None, save_prefix='',
    save_format='jpg', subset=None, interpolation='nearest',
    validate_filenames=True
)



base_model=ResNet152V2(weights=None,include_top=False, input_shape = (256,256,3)) 
#base_model.summary()


x= base_model.output



#preds=Dense(14,activation='sigmoid')(x)

#model=Model(inputs=base_model.input,outputs=preds)

#model.summary()

#model.load_weights('brucechou1983_CheXNet_Keras_0.3.0_weights.h5')

#model.summary()

#x = model.output

for layer in base_model.layers:
    layer.trainable=False

x=Flatten()(x)
#new_x  = model.output
x=Dense(512,activation='relu')(x)
x=BatchNormalization()(x)
preds=Dense(11,activation='softmax')(x)

model=Model(inputs=base_model.input,outputs=preds)
    
#model.summary()

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


def scheduler(epochs):
    learning_rate = 0.001
    tf.summary.scalar('learning rate', data=learning_rate, step=epochs)
    return learning_rate

lrscheduler = LearningRateScheduler(scheduler)
anne = ReduceLROnPlateau(monitor='lr', factor=0.1, patience=3, verbose=1, min_lr=1e-5)
checkpoint = ModelCheckpoint('model_6.h5', verbose=1, save_best_only=True, monitor='val_auc')
#ts = TensorBoard(log_dir=os.path.join(BASE_DIR, "log_resnet"))

#datagen = ImageDataGenerator(zoom_range = 0.2, horizontal_flip=True, shear_range=0.2)


#datagen.fit(X_train)
# Fits-the-model
history = model.fit(
               trainGen,
               steps_per_epoch=trainGen.n//trainGen.batch_size,
               epochs=5,
               verbose=1,
               callbacks=[checkpoint,anne,lrscheduler],
               validation_data=valGen)



#model = load_model('model_3.h5')

#model.summary()



#predict the class
#result = model.predict(test_image)
#values = prediction_to_tags(result)

#f1_score = f1(test_data,values)


In [ ]:
test_data = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/sample_submission.csv')
submission_data = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/sample_submission.csv')
test_data['StudyInstanceUID'] = test_data['StudyInstanceUID'].apply(lambda x: x +".jpg")


testGen = validationAug.flow_from_dataframe(
    test_data, directory=os.path.join(BASE_DIR,'test'), x_col='StudyInstanceUID', y_col=target_cols,
    class_mode='raw',
    weight_col=None, target_size=(224, 224), color_mode='rgb',
    classes=None, batch_size=64, shuffle=False
    )


evaluation=model.evaluate(testGen,steps=(testGen.n//testGen.batch_size)+1) 

pred_indices = model.predict(testGen,steps=(testGen.n//testGen.batch_size)+1)
prediction = pred_indices.round()

submission_data[target_cols]=prediction

submission_data.to_csv('submission.csv',index=False)